# Lab3 (Student version)

In [1]:
import random
import matplotlib.pyplot as plt
import sys
import timeit
import datetime
import math 

Download the three following graphs:
- http://lioneltabourier.fr/documents/drosophila.txt
- http://snap.stanford.edu/data/email-Eu-core.html 
- http://snap.stanford.edu/data/com-LiveJournal.html (optional, might be too large)

It is also useful to consider some toy graphs (e.g. manually created graphs with a dozen nodes) to test your programs.

## Exercise 0: preliminaries

Using the codes of Lab1, load the graphs in memory as dictionary of lists and check their number of nodes and links.

In [2]:
def read_graph_edges(filename):
    '''Just gets the edges as a list, not currenly used and replaced with "read_graph_dict"'''
    edges = set()

    with open(filename, "r") as f:
        for line in f.readlines():
            if line.startswith("#"): continue
            line = line.strip().replace("\t", " ")
            tuplee = line.split(" ")
            from_id, to_id = int(tuplee[0]), int(tuplee[1])
 
            edges.add([from_id, to_id])

    return list(edges)

def read_graph_dict(filename, undirected=True):
    '''Delete the duplicate edges and load them to memory, unidirected=True adds two way connections'''
    graph_dict = {}
    visited_edges = set()

    def register_to_dict(graph, node):
        if node not in graph:
            graph[node] = []

    def add_connection(graph, from_node, to_node):
        graph[from_node].append(to_node)

    with open(filename, "r") as f:
        for line in f.readlines():
            if line.startswith("#"): continue
            line = line.strip().replace("\t", " ")
            tuplee = line.split(" ")
            from_id, to_id = int(tuplee[0]), int(tuplee[1])

            register_to_dict(graph_dict, from_id)
            register_to_dict(graph_dict, to_id)

            tttt = tuple(sorted([from_id, to_id]))

            if tttt in visited_edges:
                continue
            elif from_id == to_id: 
                continue

            visited_edges.add(tttt)

            add_connection(graph_dict, from_id, to_id)

            if undirected:
                add_connection(graph_dict, to_id, from_id)

    return graph_dict

def number_of_edges_and_nodes(g: dict[int, list[int]]):
    '''Find number of edges and nodes'''
    node_set = set()
    edge_set = set()

    for from_node, transiton_list in g.items():
        for to_node in transiton_list:
            edge_tuple = tuple(sorted([from_node, to_node]))
            node_set.add(from_node)
            node_set.add(to_node)
            edge_set.add(edge_tuple)
    
    return len(node_set), len(edge_set)


In [3]:
g_drosophila = read_graph_dict("./drosophila.txt")
g_email = read_graph_dict("./email-Eu-core.txt")

g_drosophila_edges, g_drosophila_nodes = number_of_edges_and_nodes(g_drosophila)
print(f"g_drosophila has {g_drosophila_edges} edges and {g_drosophila_nodes} nodes")

g_email_edges, g_email_nodes = number_of_edges_and_nodes(g_email)
print(f"g_email has {g_email_edges} edges and {g_email_nodes} nodes")

g_drosophila has 7236 edges and 22270 nodes
g_email has 986 edges and 16064 nodes


## Exercise 1: BFS

### 1.1 Components

- Implement a BFS algorithm.  

- Use it on each of the graphs to evaluate the size of the largest connected component of these graphs.
- Use it to identify all connected components.

Warning: if your BFS is not well coded, it can be very long, so if it doesn't work on Amazon or LiveJournal in less than a few minutes, either improve your code, or test only on smaller graphs. 

In [4]:
def connected_components(g):
    '''Calculates the connected components. Returns a dict with node ids as keys and component id as values.'''
    node_components =  {}
    unmarked_nodes = {n for n in g.keys()}
    component_id = 0

    # BFS
    while len(unmarked_nodes) != 0:
        s_node = unmarked_nodes.pop()
        queue = [s_node]

        while len(queue) != 0:
            c_node = queue.pop(0)
            node_components[c_node] = component_id

            for adj_node in g[c_node]:
                if adj_node in unmarked_nodes: 
                    unmarked_nodes.remove(adj_node)
                    queue.append(adj_node)

        component_id += 1

    return node_components

def component_counts(node_components: dict[int, int]):
    '''Returns a dict with keys as component id and value as the number of nodes in that component.'''
    component_counts: dict[int, int] = {}
    for _, component_id in node_components.items():
        if component_id not in component_counts: # Add if component not encountered
            component_counts[component_id] = 0
        component_counts[component_id] += 1
    return component_counts


In [5]:
g_drosophila_components = connected_components(g_drosophila)
g_email_components = connected_components(g_email)

g_drosophila_component_counts = component_counts(g_drosophila_components)
g_email_component_counts = component_counts(g_email_components)

print(f"g_drosophila has total of {len(g_drosophila_component_counts)} component(s)")
print(f"g_email has total of {len(g_email_component_counts)} component(s)")
print()
print(f"g_drosophila's biggest component has {max({x for x in g_drosophila_component_counts.values()})} element(s)")
print(f"g_email's biggest component has {max({x for x in g_email_component_counts.values()})} element(s)")

g_drosophila has total of 1 component(s)
g_email has total of 20 component(s)

g_drosophila's biggest component has 7236 element(s)
g_email's biggest component has 986 element(s)


### 1.2 Distances

- Modify the BFS above to have it compute the distance to the source node.

- Using the fact that the diameter is necessarily larger than any distance measured, use your distance computation code to get a lower bound of the diameter. The higher the bound, the better.

In [6]:
def modified_bfs(g: dict[int, list[int]], s: int):
    '''Modifies version of bfs, returnes the distance dict from s to u where u is the key of the dict.'''
    #F is the queue, s is the starting node, visited stands for marked
    visited = set()
    F = [s]
    component = set()
    
    # In some cases there can be edges not located in the graph's keys but in the connection list.
    # The given graph assumed to not satisfy this property.
    # dist = {node: -1 for node in g}
    dist = {}
    
    #dist=[-1] * len(graph)
    dist[s] = 0

    while F:
        node = F.pop(0)
        visited.add(node)
        component.add(node)

        for neighbor in g[node]:
            distance_neigbor = -1

            if neighbor in dist:
                distance_neigbor = dist[neighbor]

            if neighbor not in visited and neighbor not in F and distance_neigbor == -1:
                F.append(neighbor)
                dist[neighbor] = dist[node] + 1

    return dist


#find the largest component in the graph
def maximal_connected_component(g):
    '''maximal connected component algorithm'''
    visited = set()
    maximal_component = set()

    for node in g:
        if node not in visited:
            component = modified_bfs(g, node)
            visited.update(component)
            if len(component)>len(maximal_component):
                maximal_component=component

    return maximal_component

def approximate_diameters(g, iterations=1):
    '''Approximates the diameter, default number of iterations is 1.'''
    MC=maximal_connected_component(g)
    # print(f"maximal connected component: {MC}")
    diameters=[]

    # print(len(MC))
    
    #For each node v in the MC, compute a BFS from v to obtain all the distances form v to every other node in the MC
    for node in list(MC)[:iterations]: # do N iterations
        distances=modified_bfs(g, node)
        # print("sdlmsdsod")
        #Everytime we compute a BFS from a node v, keep the max_distance obtained with the BFS.
        max_dist=max(distances.values())
        #At the end of every BFS from v, add max_distance to the list of diameters
        diameters.append(max_dist)
        #Repeat for every node in the MC.
    
    return diameters


In [7]:
# Here the iteration is not set which means only one iteration performed. From our tests this is usually good enough.
g_drosophila_diameters=approximate_diameters(g_drosophila) # Takes too long? (two of the takes 30 seconds in Ufuk's laptop)
print(f"g_diameters lower bound on the diameters is {max(g_drosophila_diameters)}")

g_email_diameters=approximate_diameters(g_email) 
print(f"g_email lower bound on the diameters is {max(g_email_diameters)}")

g_diameters lower bound on the diameters is 8
g_email lower bound on the diameters is 4


In [8]:
# Here the iteration is set to 100, which means it does the diameter calculation on 100 nodes to find the largest distance.
g_drosophila_diameters=approximate_diameters(g_drosophila, iterations=100)
print(f"g_diameters lower bound on the diameters is {max(g_drosophila_diameters)}")

g_email_diameters=approximate_diameters(g_email, iterations=100) 
print(f"g_email lower bound on the diameters is {max(g_email_diameters)}")

g_diameters lower bound on the diameters is 10
g_email lower bound on the diameters is 5


## Exercise 2: Triangles

### 2.1 Raw triangle counting

- Implement the 2 triangle counting algorithms presented in the course. 

- Test your program on the 3 graphs and report the number of triangles as well as the running time of your program.

In [9]:
#NAIVE ALGORITHM-computing the number of triangles
def naive_count_triangles(g):
    '''Counts the triangles naively.'''
    count=0
    for node in g:
        for n1 in g[node]:
            for n2 in g[node]:
                if n2!=n1:
                    if n1 in g[n2]:
                        count+=1
    return int(count/6)

#IMPROVED ALGORITHM-computing the number of triangles
def improved_count_triangles(g):
    '''Uses the improved version of triangle count'''
    count=0

    for n1, neighbors in g.items():
        for n2 in neighbors:
            if n1 < n2:
                for n3 in set(g[n2]) & set(g[n1]):
                    if n2 < n3:
                        count+=1
    return count // 1

def time(funcion, *args, **kwargs):
    '''Runs the given function with the additional arguments and reports the time using pring. Also returns the variable returned by the function.'''
    before = datetime.datetime.now()
    r = funcion(*args, **kwargs)
    after = datetime.datetime.now()

    print(f"Function call took {(after - before).total_seconds()} second(s)")

    return r


In [10]:

g_drosophila_numtriangles = time(naive_count_triangles, g_drosophila)
g_email_numtriangles = time(naive_count_triangles, g_email)

print(f"g_drosophila has {g_drosophila_numtriangles} triangle(s)")
print(f"g_email has {g_email_numtriangles} triangle(s)")

g_drosophila_numtriangles = time(improved_count_triangles, g_drosophila)
g_email_numtriangles = time(improved_count_triangles, g_email)

print(f"g_drosophila has {g_drosophila_numtriangles} triangle(s)")
print(f"g_email has {g_email_numtriangles} triangle(s)")


Function call took 0.197691 second(s)
Function call took 1.217784 second(s)
g_drosophila has 2179 triangle(s)
g_email has 105461 triangle(s)
Function call took 0.037564 second(s)
Function call took 0.100978 second(s)
g_drosophila has 2179 triangle(s)
g_email has 105461 triangle(s)


### 2.2 Transitive ratio

Use this program to compute the transitive ratio of the graphs. Remember that the transitive ratio is defined as 
$$ \frac{3.number \ of \ triangles}{number \ of \ forks}$$
and that the number of forks (or connected triples) of a node of degree $d$ is simply $\frac{d(d-1)}{2}$.

In [11]:
def transitive_ratio(g: dict[int, list[int]]):
    number_of_forks = 0
    for from_node, connecction_list in g.items():
        d = len(connecction_list) # out-degree of the node
        number_of_forks += d * (d - 1) / 2
    number_of_triangles = improved_count_triangles(g)

    return 3 * number_of_triangles / number_of_forks

In [12]:
g_drosophila_transitive_ratio = transitive_ratio(g_drosophila)
g_email_transitive_ratio = transitive_ratio(g_email)

print(f"g_drosophila has the transitive ratio {g_drosophila_transitive_ratio}")
print(f"g_email has the transitive ratio {g_email_transitive_ratio}")

g_drosophila has the transitive ratio 0.014255744168600289
g_email has the transitive ratio 0.26739242877040204


## * Exercice 3: Recent Research

\* Star exercice is a little bit advanced but very enriching. The point of this exercice is a little introduction to research paper reading and implementing some simple algorithms. 

If you do not do this exercice at all and perfectly do the other 2 (Exercice 1 and 2) you will get a good grade. But doing this exercice even only a part of it will help you get the highest marks.

**I do not want you to understand all the content of the papers. We are just implementing given algorithms. But of course understanding research papers can help you in your future career.**

**Do not hesitate to contact me or email me if you need any help.**

### 3.1  Recent research on triangle counting:

Take a look at the recent article (published in [Alenex 2023](https://epubs.siam.org/doi/book/10.1137/1.9781611977561)
) : https://epubs.siam.org/doi/pdf/10.1137/1.9781611977561.ch7 and implement the algorithms A++ and A+- and compare their running times on the 3 graphs

In [13]:
def a_plus_plus(g: dict[int, list[int]]):
    '''Implemented from the given paper, finds the num of triangles in the graph by in-coming nodes.'''
    g_reverse = {node: [] for node in g.keys()}
    
    # B = {v: False for v in g.keys()}
    # Instead of a dict we used a set for this implementation to represent v: True
    B = set()

    num_triangles = 0

    for from_node, connection_list in g.items():
        for to_node in connection_list:
            g_reverse[to_node].append(from_node)
    

    for w in g.keys():
        current_out_nodes = g[w]
        current_in_nodes = g[w]
        
        # Set the incoming nodes in the dict B true
        B = {v for v in current_in_nodes}

        for u in current_in_nodes:
            u_outgoing_nodes = g[u]
            for v in u_outgoing_nodes:
                if v in B:
                    # triangle pair is (w, u, v)
                    num_triangles += 1 # we found a triangle

        B.clear()
    
    return num_triangles // 6

def a_plus_minus(g: dict[int, list[int]]):
    '''Implemented from the given paper, finds the num of triangles in the graph by out-going nodes.'''
    g_reverse = {node: [] for node in g.keys()}
    
    # B = {v: False for v in g.keys()}
    # Instead of a dict we used a set for this implementation to represent v: True
    B = set()

    num_triangles = 0

    for from_node, connection_list in g.items():
        for to_node in connection_list:
            g_reverse[to_node].append(from_node)
    

    for u in g.keys():
        current_out_nodes = g[u]
        current_in_nodes = g[u]
        
        # Set the incoming nodes in the dict B true
        B = {v for v in current_out_nodes}

        for v in current_out_nodes:
            v_outgoing_nodes = g[v]
            for w in v_outgoing_nodes:
                if w in B:
                    # triangle pair is (w, u, v)
                    num_triangles += 1 # we found a triangle

        B.clear()
    
    return num_triangles // 6

In [14]:
g_drosophila_numtriangles = time(a_plus_plus, g_drosophila)
g_email_numtriangles = time(a_plus_plus, g_email)

print(f"g_drosophila has {g_drosophila_numtriangles} triangle(s)")
print(f"g_email has {g_email_numtriangles} triangle(s)")

g_drosophila_numtriangles = time(a_plus_minus, g_drosophila)
g_email_numtriangles = time(a_plus_minus, g_email)

print(f"g_drosophila has {g_drosophila_numtriangles} triangle(s)")
print(f"g_email has {g_email_numtriangles} triangle(s)")

Function call took 0.040778 second(s)
Function call took 0.090444 second(s)
g_drosophila has 2179 triangle(s)
g_email has 105461 triangle(s)
Function call took 0.040754 second(s)
Function call took 0.091602 second(s)
g_drosophila has 2179 triangle(s)
g_email has 105461 triangle(s)


### 3.2  Recent research on Clustering Coefficient:

Take a look at the recent article (published in [LATIN 2022](https://pakal.cs.cinvestav.mx/latin2022/)
) : https://www.inf.ufpr.br/vignatti/v/papers/2022-latin.pdf and implement the algorithm of local clustering coefficient estimation with relative errors of $ \epsilon = 1, 0.5, 0.1$ with probability $ p = 0.25$ and $\delta = 0.8$

In [15]:
def local_cluster_coeff(g: dict[int, list[int]], c_aps, epsilon, p, delta):
    '''At least we tried'''
    g_reverse = {node: [] for node in g.keys()}
    E = set()
    T = {v: 0 for v in g.keys()}

    for from_node, connection_list in g.items():
        for to_node in connection_list:
            g_reverse[to_node].append(from_node)
            E.add((from_node, to_node))

    m = len(E)

    node_degrees = {n: len(t) for n, t in g.items()}

    for n, t in g_reverse.items():
        node_degrees[n] += len(t)

    # node_degrees naps the node to the degree where degree := in-degree + out-degree.
    # Since we actually assume graphs to be undirected, we are counting the degrees twice,
    # thus we might want to 

    big_delta = max(node_degrees.items(), key=lambda x: x[1])[0]

    temp_line = (math.floor(math.log(big_delta) - 1) + 1) * math.log10(1/p) + math.log(1 / delta)
    r = math.ceil((c_aps / (p * epsilon**2)) * temp_line)

    for i in range(1, r):
        if not E: break # break if there are no more edges left
        a, b = E.pop()
        N_a = set(g[a]).union(set(g_reverse[a]))
        N_b = set(g[b]).union(set(g_reverse[b]))
        for v in N_a:
            if v in N_b:
                T[v] = T[v] + m / r

    l = {v: 2 * T[v] / (epsilon + node_degrees[v] * (1 - node_degrees[v])) for v in g.keys()}

    return l

In [16]:
print(local_cluster_coeff(g_drosophila, 100, 1.0, 0.25, 0.8)) # example output of the last implementation
print(local_cluster_coeff(g_drosophila, 100, 0.5, 0.25, 0.8))
print(local_cluster_coeff(g_drosophila, 100, 0.1, 0.25, 0.8))

print(local_cluster_coeff(g_email, 100, 1.0, 0.25, 0.8))
print(local_cluster_coeff(g_email, 100, 0.5, 0.25, 0.8))
print(local_cluster_coeff(g_email, 100, 0.1, 0.25, 0.8))

{0: -0.0, 1: -0.02693829141437219, 2: -0.0, 3: -0.006185713335378572, 4: -0.0, 5: -0.0, 6: -0.0, 7: -0.0, 8: -0.0, 9: -0.0, 10: -0.0, 11: -0.045339717042433454, 12: -0.0, 13: -0.004681296365946628, 14: -0.012745936624593248, 15: -0.022202234617029835, 16: -0.0, 17: -0.0, 18: -0.009240789794992365, 19: -0.0, 20: -0.0, 21: -0.0, 22: -0.0, 23: -0.0, 24: -0.0, 25: -0.01798584028985712, 26: -0.0, 27: -0.009973314554064198, 28: -0.0, 29: -0.0, 30: -0.0, 31: -0.0, 32: -0.0, 33: -0.0, 34: -0.0, 35: -0.0, 36: -0.010366029771032695, 37: -0.0, 38: -0.0, 39: -0.0, 40: -0.0, 41: -0.0, 42: -0.0, 43: -0.0, 44: -0.0, 45: -0.0, 46: -0.0, 47: -0.0, 48: -0.014722681242436956, 49: -0.0, 50: -0.0, 51: -0.0, 52: -0.0, 53: -0.0, 54: -0.0, 55: -0.007549161499005168, 56: -0.0, 57: -0.0, 58: -0.0, 59: -0.003343247229829932, 60: -0.0, 61: -0.0, 62: -0.03640444405575258, 63: -0.0, 64: -0.009327390245840814, 65: -0.0, 66: -0.0, 67: -0.0, 68: -0.0, 69: -0.0, 70: -0.0, 71: -0.0, 72: -0.0, 73: -0.0008687523341379858,